In [ ]:
import os
os.chdir(r'C:\Users\jf79\Repos\REAP-Analytics')

import myfuncs.myfuncs as mf
mf.read_directory()

import pandas as pd
pd.set_option('display.max_columns', None)

import re
import warnings
warnings.filterwarnings('ignore')

In [ ]:
customer_columns = {
    'InteractionsKey':'interactionskey',
    'ETLTaskID':'etltaskid',
    'RowNumber':'rownumber',
    'GroupName':'groupname',
    'Type':'type',
    'Direction':'direction',
    'TimeQueued':'timequeued',
    'TimeEnded':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Username':'username',
    'UserID':'userid',
    'CustomerName':'customername',
    'CustomerContact':'customercontact',
    'QueueWait':'queuetime',
    'Duration':'duration',
    'TalkTime':'talktime',
    'CallHoldDuration':'holdtime',
    'WrapUpDuration':'wrapupduration',
    'AgentHandlingTime':'agenthandlingtime',
    'Result':'result',
    'ActivityCode':'activitycode',
    'AssistedBy':'assistedby',
    'TransferredTo':'transferredto',
    'MediaInteractionId':'interactionid',
    'Subject':'subject',
    'CaseReference':'casereference',
    'Tags':'tags',
    'Forwarded':'forwarded',
    'IPAddress':'ipaddress',
    'FormattedSource':'formattedsource',
    'ConversationID':'conversationid',
    'ConversationStartTime':'conversationstarttime',
    'ConversationEndTime':'conversationendtime',
    'ConversationDuration':'conversationduration'
}
master_columns = {
    'Interaction Id':'interactionid',
    'Type':'type',
    'Direction':'direction',
    'Time Queued':'timequeued',
    'Time Ended':'timeended',
    'Queue':'queue',
    'Skill':'skill',
    'Agent':'agent',
    'Customer Name':'customername',
    'Customer Contact':'customercontact',
    'Queue Time':'queuetime',
    'Duration':'duration',
    'Talk Time':'talktime',
    'Hold Time':'holdtime',
    'Wrap-up Duration':'wrapupduration',
    'Agent Handling Time':'agenthandlingtime',
    'Result':'result',
    'Activity Code':'activitycode',
    'Assisted By':'assistedby',
    'Transferred To':'transferredto',
    'Subject':'subject',
    'Conversation Start Time':'conversationstarttime',
    'Conversation End Time':'conversationendtime',
    'Conversation Duration':'conversationduration',
    'Agent Username':'agentusername',
    'Agent ID':'agentid',
    'Case Reference':'casereference',
    'Forwarded':'forwarded',
    'Source':'source',
    'Conversation ID':'conversationid',
    'IP Address':'ipaddress'
}

In [ ]:
customer_service_data = mf.query_data(schema='Netcall', data='Interactions')

In [ ]:
path_to_data = r"C:\Users\jf79\OneDrive - Office Shared Service\Sharepoint Links\BI - Corporate - Documents\REAP\1. REAP Monthly Report 2025\Netcall\All Historical Data\Interactions Data"
os.chdir(path_to_data)

mf.read_directory()

# Create an empty DataFrame to store the combined data
master_data = pd.DataFrame()
 
# Iterate over each folder in the root directory
for folder in os.listdir(path_to_data):
    folder_path = os.path.join(path_to_data, folder)
    # Ensure it's a directory
    if os.path.isdir(folder_path):
        # Iterate over each CSV file in the folder
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):  # Ensure it's a CSV file
                file_path = os.path.join(folder_path, file)
 
                # Read CSV
                df = pd.read_csv(file_path)
 
                # Add a new column with the CSV filename
                df["Source_File"] = file
 
                # Append to master DataFrame
                master_data = pd.concat([master_data, df], ignore_index=True)

In [ ]:
customer_service = customer_service_data.copy()
customer_service.rename(
    columns=customer_columns,
    inplace=True
)
customer_service = customer_service[customer_service['groupname'] == 'Customer_Services']

columns_to_drop = [
    'interactionskey','etltaskid','rownumber',
    'username','userid','tags','formattedsource'
]
customer_service.drop(columns=columns_to_drop, inplace=True)
col = customer_service.pop('interactionid')
customer_service.insert(0, 'interactionid', col)
customer_service['groupname'] = customer_service['groupname'].str.replace('Customer_Services','Customer Services')

customer_service.sort_values(
    ['timequeued'],
    ascending=True,
    inplace=True
)

drop = customer_service[customer_service['interactionid'].duplicated(keep='first')]
customer_service.drop(
    drop.index,
    inplace=True
)
customer_service.reset_index(
    drop='index',
    inplace=True
)

customer_service['timequeued'] = pd.to_datetime(customer_service['timequeued'], format='mixed', dayfirst=True)
customer_service['timeended'] = pd.to_datetime(customer_service['timeended'], format='mixed', dayfirst=True)
customer_service['Year'] = customer_service['timequeued'].dt.year
customer_service['Month'] = customer_service['timequeued'].dt.month

In [ ]:
master_df = master_data.copy()
master_df.rename(
    columns=master_columns,
    inplace=True
)

# Step 1: Split at the first dot (.)
master_df[['groupname', 'to_drop']] = master_df['Source_File'].str.split('.', expand=True)
 
# Step 2: Split at the first opening parenthesis (()
master_df[['groupname', 'to_drop']] = master_df['groupname'].str.split('(', expand=True)
 
# Step 3: Clean text (remove punctuation & normalize spaces)
master_df['groupname'] = master_df['groupname'].apply(lambda text: re.sub(r"[^\w\s]", "", text).strip())
master_df['groupname'] = master_df['groupname'].str.replace('Childrens', 'Children')
master_df['groupname'] = master_df['groupname'].str.replace('Children', 'Childrens')
master_df['groupname'] = master_df['groupname'].str.replace('ICAT6600', 'ICAT 6600')
master_df['groupname'] = master_df['groupname'].str.replace('Refugee', 'Refuge')
master_df['groupname'] = master_df['groupname'].str.replace('Refuge', 'Refugee')
master_df['groupname'] = master_df['groupname'].str.replace('Netcall HMS', 'HMS')
master_df['groupname'] = master_df['groupname'].str.replace('HMS', 'Netcall HMS')
master_df['groupname'] = master_df['groupname'].str.replace('Recovery Queue', 'Recovery')

col = master_df.pop('interactionid')
master_df.insert(0, 'interactionid', col)
col = master_df.pop('groupname')
master_df.insert(1, 'groupname', col)

master_df.sort_values(
    ['timequeued','agentid'],
    ascending=True,
    inplace=True
)

master_df.drop(
    columns=['Tags','Source_File','to_drop','Group'],
    inplace=True
)
drop = master_df[master_df['interactionid'].duplicated(keep='first')]
master_df.drop(
    drop.index,
    inplace=True
)
master_df.reset_index(
    drop='index',
    inplace=True
)


master_df['timequeued'] = pd.to_datetime(master_df['timequeued'], format='mixed', dayfirst=True)
master_df['timeended'] = pd.to_datetime(master_df['timeended'], format='mixed', dayfirst=True)
master_df['Year'] = master_df['timequeued'].dt.year
master_df['Month'] = master_df['timequeued'].dt.month

In [ ]:
final_df = pd.concat([master_df,customer_service])

values_to_change = [
    'Unknown','Completed','0'
]
for value in values_to_change:
    if value == '0':
        final_df['result'] = final_df['result'].str.replace('0','Failed')
    else:
        final_df['result'] = final_df['result'].str.replace(f'{value}','Other')

final_df['nullcount'] = final_df.isna().sum(axis=1)
final_df.sort_values(by=['interactionid','nullcount'], inplace=True)
drop = final_df[final_df['interactionid'].duplicated(keep='first')]
final_df.drop(drop.index, inplace=True)
final_df.drop(columns='nullcount', inplace=True)

In [ ]:
check = mf.id_check(final_df, 'interactionid', keep='first', directory=path_to_data)
mf.display(check)

if check is not None:
    final_df = final_df[~final_df['interactionid'].isin(check['interactionid'])]
    mf.display(final_df)

In [ ]:
mf.export_file(
    final_df, 
    directory = path_to_data, 
    df_name='Netcall Interactions Master Data',
    file_type='csv'
)